In [39]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [40]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

In [51]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def movie_data_import():
    
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}ratings.csv')


    # Open and read the Wikipedia data JSON file.
    f'{file_dir}wikipedia-movies.json'
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
        wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    return wiki_movies_df, kaggle_metadata, ratings


    # Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]     
    

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies_raw]     
    

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)


    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    # dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e: 
        print("The error raised is: ", e) 

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]    
    

    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()

    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    
    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

    
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)' 
    

    # Add the parse_dollars function.
    box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    box_office.str.extract(f'({form_one}|{form_two})')
    def parse_dollars(s):
        if type(s) != str:
            return np.nan
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**6
            return value
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**9
            return value
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value
        else:
            return np.nan    
        
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)

    
    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
    budget[~matches_form_one & ~matches_form_two]
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    budget[~matches_form_one & ~matches_form_two]
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)     
    

    # Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    

    # Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
     
    # 2. Clean the Kaggle metadata.
    kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] == 'True'
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
    ratings.info(null_counts=True)
    pd.to_datetime(ratings['timestamp'], unit='s')
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

    
    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])


    # 4. Drop unnecessary columns from the merged DataFrame.
   

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column], axis=1)
        df.drop(columns=wiki_column, inplace=True)


    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
    

    # 7. Filter the movies DataFrame for specific columns.
    for col in movies_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
    if num_values == 1:
        

    # 8. Rename the columns in the movies DataFrame.
        movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]
        movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)


    # 9. Transform and merge the ratings DataFrame.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [52]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = 'C://Users/drchr/Class/Movies-ETL/'

# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'

# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'

# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [53]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = movie_data_import()

In [54]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [56]:
 wiki_movies_df.drop(['Directed by', 'Produced by',  'Screenplay by',  'Story by',  'Narrated by',  'Music by',  'Edited by',  'Productioncompany ',
    'Distributed by',  'Written by',  'Genre',  'Theme music composer',  'Country of origin',  'Original language(s)',
    'Producer(s)',  'Production company(s)',  'Original network',  'Original release',  'Productioncompanies ',
    'Executive producer(s)',  'Production location(s)',  'Picture format',  'Audio format',  'Voices of',  'Followed by',
    'Created by',  'Also known as',  'Opening theme',  'No. of episodes',  'Preceded by',  'Author',  'Publisher',
    'Publication date',  'Media type',  'Pages',  'ISBN',  'OCLC',  'LC Class',  'Cover artist',  'Series',  'Set in',  'Adaptation by',
    'Suggested by',  'Biographical data',  'Born',  'Died',  'Resting place',  'Occupation',  'Years active',  'Spouse(s)',
    'Children',  'Parent(s)',  'Genres',  'Instruments',  'Labels',  'Website',  'Traditional',  'Mandarin',  'Type',  'Industry',
    'Fate', 'Founded',  'Founder',  'Headquarters',  'Parent',  'Recorded',  'Venue',  'Length',  'Label',  'Area',  'Coordinates',
    'Status',  'Opening date',  'Closing date',  'Replaced',  'Replaced by',  'Name',  'Attraction type',  'Music',  'Duration',
    'Simplified Chinese',  'Traditional Chinese',  'Hanyu Pinyin',  'Literal meaning',  'Transcriptions',  'Bopomofo',
    'Gwoyeu Romatzyh',  'Wade–Giles',  'IPA',  'Yale Romanization',  'Jyutping',  'Hokkien POJ',  'Animation by',
    'Color process',  'Engine(s)',  'Genre(s)',  'Actor control',  'Release(s)',  'Format(s)',  'Simplified',  'Characters',
    'Date premiered', 'Place premiered',  'Setting',  'Original language',  'Subject',  'Published',  'Dewey Decimal',
    'Text',  'Illustrator',  'Original title',  'Published in English',  'French',  'Developed by',  'Ending theme',  'No. of seasons',
    'Nationality',  'Portrayed by',  'Alias',  'Species',  'Gender',  'Family',  'Alma mater',  'Camera setup',  'Novel(s)',  'Comics',
    'Film(s)',  'Screen story by',  'Hangul',  'Revised Romanization',  'McCune–Reischauer',  'Developer(s)',  'Publisher(s)',
    'Designer(s)',  'Programmer(s)',  'Artist(s)',  'Engine',  'Platform(s)',  'Release',  'Mode(s)',  'Original work',  'Television series',
    'Japanese',  'Hepburn',  'Literally',  'Cantonese',  'Full name',  'Height',  'Seasons',  'Chinese',  'Other names',  'Relatives',
    'Yiddish',  'Formerly',  'Key people',  'Total assets',  'Owner',  'Number of employees',  'Divisions',  'Subsidiaries',
    'Arabic',  'Romanized',  'Predecessor',  'Founders',  'Area served',  'Products',  'Services',  'Russian',  'Hebrew',
    'Revenue',  'Operating income',  'Polish'], axis=1, inplace=True)
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Editor(s)',
 'Distributor',
 'Composer(s)',
 'Released',
 'Director',
 'Producer',
 'Production company',
 'Writer(s)']

In [57]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df = wiki_movies_df.reindex(columns=['url', 'year', 'imdb_link', 'title', 'Based on', 'Starring', 'Cinematography', 'Release date', 'Country', 'Language', 'Budget', 'Director', 'Distributor', 'Editor(s)', 'Composer(s)', 'Producer(s)', 'Production company(s)', 'Writer(s)', 'imdb_id', 'box_office', 'budget', 'release_date', 'running_time'])
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
movies_with_ratings_df.columns.to_list()

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [62]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df = movies_with_ratings_df.reindex(columns=['imdb_id',  'id', 'title', 'original_title', 'adult',  'tagline',  'belongs_to_collection',  'homepage', 'budget',  'genres',  'original_language',   'overview',
 'popularity',  'poster_path',  'production_companies',  'production_countries',  'release_date',  'revenue',  'runtime',  'spoken_languages',
 'status',  'video',  'vote_average',  'vote_count'])
movies_with_ratings_df.head()

,imdb_id,id,title,original_title,adult,tagline,belongs_to_collection,homepage,budget,genres,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,video,vote_average,vote_count
0,tt0114709,862,Toy Story,Toy Story,FALSE,NaN,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",...,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,False,7.7,5415.0
1,tt0113497,8844,Jumanji,Jumanji,FALSE,Roll the dice and unleash the excitement!,NaN,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,False,6.9,2413.0
2,tt0113228,15602,Grumpier Old Men,Grumpier Old Men,FALSE,Still Yelling. Still Fighting. Still Ready for...,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,False,6.5,92.0
3,tt0114885,31357,Waiting to Exhale,Waiting to Exhale,FALSE,Friends are the people who let you be yourself...,NaN,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",...,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,False,6.1,34.0
4,tt0113041,11862,Father of the Bride Part II,Father of the Bride Part II,FALSE,Just When His World Is Back To Normal... He's ...,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,0,"[{'id': 35, 'name': 'Comedy'}]",...,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,False,5.7,173.0


In [63]:
movies_df.columns.to_list()

['userId', 'movieId', 'rating', 'timestamp']

In [33]:
# 15. Check the movies_df DataFrame. 
movies_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
